  # Lab: Obtener salidas estructuradas de la IA Generativa


  ### Objetivos del Lab

  - Conseguir salidas estructuradas con Pydantic

  - Dominar tool calling para crear asistentes con herramientas externas

  - Integrar bases de datos con SQLAlchemy para persistencia de datos

  - Construir un asistente financiero con notas persistentes



  ### Tecnologías que exploraremos

  - **Pydantic**: Para definir esquemas de respuesta estructurada

  - **Tool Calling**: Para crear funciones que el modelo puede ejecutar

  - **SQLAlchemy**: Para gestión de bases de datos relacionales

  - **Azure OpenAI**: Como plataforma de IA principal


  ### Estructura del Lab

  1. **Preparación del entorno**

  2. **El problema del JSON manual**

  3. **Structured Output con Pydantic- La solución**

  4. **Tool Calling**

  5. **Bases de datos con SQLAlchemy**

  6. **Integración: Structured Output + Base de datos y Tool Calling + Base de datos**

  7. **Problema: Asistente Financiero con Notas**

  ## 1. Preparación del Entorno







  ### Instalación de dependencias







  Ejecuta la siguiente celda para instalar todas las librerías necesarias:

In [ ]:
# Instalar todas las dependencias necesarias para el lab
# Sqlite ya viene instalado con Python
!pip install python-dotenv openai pydantic sqlalchemy


  ### Estructura de archivos necesaria


  1. **Archivo `.env`** (para credenciales Azure OpenAI)

  ### ⚠️ Configuración requerida

  Necesitarás configurar las siguientes variables de entorno en tu archivo `.env`:



  ```

  AZURE_OPENAI_API_KEY=tu_api_key

  AZURE_OPENAI_ENDPOINT=https://tu-resource.openai.azure.com/

  AZURE_OPENAI_DEPLOYMENT_NAME=tu_deployment_name

  ```

  ## 2. El problema con JSON "manual"



  ### 2.1 JSON Simple

In [ ]:
from openai import AzureOpenAI
import json
import os 
from pydantic import BaseModel


class PersonaSimple(BaseModel):
    nombre: str
    edad: int


def intentar_json_simple():
    """JSON simple funciona consistentemente"""
    
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-02-01"
    )
    
    prompt = """
    Inventa una persona y devuelve un JSON con su nombre y edad: {"nombre": "string", "edad": numero}
    Solo devuelve el JSON, sin texto adicional ni explicaciones.
    """
    
    print("✅ JSON SIMPLE")
    print("=" * 30)
    
    for intento in range(5):
        try:
            response = client.chat.completions.create(
                model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
                messages=[{"role": "user", "content": prompt}],
                temperature=1.0
            )
            
            resultado = response.choices[0].message.content.strip()
            print(f"Respuesta del modelo: {resultado}")
            data_dict = json.loads(resultado)
            persona = PersonaSimple(**data_dict)
            print(f"✅ Intento {intento + 1}: {persona.nombre}, {persona.edad} años")
            
        except Exception as e:
            print(f"❌ Intento {intento + 1}: {e}")

intentar_json_simple()


  ### 2.2 JSON Complejo - Aquí empiezan los problemas

In [ ]:
import os
import json
from typing import List
from pydantic import BaseModel
from dotenv import load_dotenv
from openai import AzureOpenAI
from enum import Enum

# Cargar variables de entorno
load_dotenv()


class TipoTrabajo(str, Enum):
    TIEMPO_COMPLETO = "tiempo_completo"
    MEDIO_TIEMPO = "medio_tiempo"
    FREELANCE = "freelance"
    ESTUDIANTE = "estudiante"
    DESEMPLEADO = "desempleado"

class NivelIdioma(str, Enum):
    BASICO = "basico"
    INTERMEDIO = "intermedio"
    AVANZADO = "avanzado"
    NATIVO = "nativo"

class Idioma(BaseModel):
    nombre: str
    nivel: NivelIdioma

class ContactoPersona(BaseModel):
    email: str
    telefono: str
    ciudad: str
    codigo_postal: int

class PerfilPersonaComplejo(BaseModel):
    nombre_completo: str
    edad: int
    contacto: ContactoPersona  # Objeto anidado
    tipo_trabajo: TipoTrabajo  # Enum
    idiomas: List[Idioma]  # Lista de objetos
    hobbies: List[str]  # Lista simple
    puntuaciones_tests: List[float]  # Lista de números
    tiene_mascotas: bool
    anos_experiencia: int
    salario_esperado: float

def intentar_json_manual():
    """Muestra los problemas de generar JSON sin structured output"""
    
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-02-01"
    )
    
    prompt = """Crea un perfil complejo de una persona llamada María en este formato JSON exacto:
    {
        "nombre_completo": "string",
        "edad": numero_entero,
        "contacto": {
            "email": "email_valido",
            "telefono": "string",
            "ciudad": "string",
            "codigo_postal": numero_entero
        },
        "tipo_trabajo": "tiempo_completo|medio_tiempo|freelance|estudiante|desempleado",
        "idiomas": [
            {
                "nombre": "string",
                "nivel": "basico|intermedio|avanzado|nativo"
            }
        ],
        "hobbies": ["hobby1", "hobby2"],
        "puntuaciones_tests": [numero_decimal1, numero_decimal2],
        "tiene_mascotas": true|false,
        "anos_experiencia": numero_entero,
        "salario_esperado": numero_decimal
    }
    
    IMPORTANTE: Responde SOLO con JSON válido, sin explicaciones ni markdown."""
    
    print("🔍 INTENTANDO GENERAR JSON SIN STRUCTURED OUTPUT")
    print("=" * 55)
    
    # Hacer varios intentos para mostrar inconsistencias
    for intento in range(4):
        try:
            response = client.chat.completions.create(
                model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
                messages=[{"role": "user", "content": prompt}],
                temperature=1.0  # Más creatividad = más problemas
            )
            
            resultado = response.choices[0].message.content.strip()
            print(f"\n🔄 Intento {intento + 1}:")
            print("📤 Respuesta cruda del modelo:")
            print(f"'{resultado}'")
            
            # Intentar parsear como JSON
            try:
                data_dict = json.loads(resultado)
                print("✅ JSON parseado correctamente")
                
                # Intentar crear la clase Pydantic compleja
                try:
                    perfil = PerfilPersonaComplejo(**data_dict)
                    print("✅ ¡Clase Pydantic creada exitosamente!")
                    print(f"   - Nombre: {perfil.nombre_completo}")
                    print(f"   - Edad: {perfil.edad}")
                    print(f"   - Idiomas: {len(perfil.idiomas)} elementos")
                    
                except Exception as e:
                    print(f"❌ Error creando clase Pydantic: {e}")

            except json.JSONDecodeError as e:
                print(f"❌ JSON INVÁLIDO: {e}")
                
                # Diagnosticar problemas comunes
                if resultado.startswith("```"):
                    print("   🔍 Problema: El modelo agregó markdown")
                if any(palabra in resultado.lower() for palabra in ["aquí", "ejemplo", "perfil"]):
                    print("   🔍 Problema: El modelo agregó texto explicativo")
                if resultado.count('{') != resultado.count('}'):
                    print("   🔍 Problema: Llaves desbalanceadas")
                
        except Exception as e:
            print(f"❌ Error general en intento {intento + 1}: {e}")
    
# Ejecutar demostración
intentar_json_manual()


  ### 🚨 Problemas típicos que aparecen:

  Al intentar generar JSON complejo manualmente, estos son los errores más comunes:

  ❌ **Respuestas con markdown** (```json)

  ❌ **Texto explicativo extra** antes o después del JSON

  ❌ **Tipos incorrectos** 

  ❌ **Campos faltantes** o con nombres ligeramente diferentes

  ❌ **JSON malformado** (comillas desbalanceadas)


  ## 3. Structured Output con Pydantic - La Solución


  ### ¿Porque Structured Output?


  **Structured Output** resuelve TODOS los problemas anteriores:

  - **JSON puro**: Sin markdown ni texto extra

  - **Tipos correctos**: Automáticamente validados

  - **Campos garantizados**: Nunca faltan campos requeridos

  - **Validación automática**: Compatible con Pydantic siempre


  📚 **Documentación útil:**

  - [Pydantic Documentation](https://docs.pydantic.dev/)

  - [Azure OpenAI Structured Output](https://learn.microsoft.com/es-es/azure/ai-foundry/openai/how-to/structured-outputs?tabs=python-secure%2Cdotnet-entra-id&pivots=programming-language-python)

  - [Openai Structured Output](https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat)

In [ ]:

def usar_structured_output():
    
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-08-01-preview"  
    )
    
    prompt = "Crea un perfil completo de una persona llamada María"
    
    print("✅ USANDO STRUCTURED OUTPUT CORRECTAMENTE")
    print("=" * 50)
    
    try:
        for intento in range(4):
            response = client.chat.completions.parse(
                model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
                messages=[{"role": "user", "content": prompt}],
                response_format=PerfilPersonaComplejo
            )
            
            # El modelo SIEMPRE devuelve un objeto Pydantic válido
            perfil = response.choices[0].message.parsed
            print("📤 Objeto Pydantic devuelto directamente:")
            print(f"Tipo: {type(perfil)}")
            
            print("\n✅  ¡Objeto Pydantic complejo creado directamente!")
            print(f"   👤 Nombre: {perfil.nombre_completo}")
            print(f"   🎂 Edad: {perfil.edad} años") 
            print(f"   📧 Email: {perfil.contacto.email}")
            print(f"   🏙️ Ciudad: {perfil.contacto.ciudad}")
            print(f"   💼 Tipo trabajo: {perfil.tipo_trabajo.value}")
            print(f"   🗣️ Idiomas: {len(perfil.idiomas)} idiomas")
            for idioma in perfil.idiomas:
                print(f"      - {idioma.nombre}: {idioma.nivel.value}")
            print(f"   🎯 Hobbies: {', '.join(perfil.hobbies)}")
            print(f"   💰 Salario esperado: {perfil.salario_esperado}€")
            
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Ejemplo simple de comparación
def comparar_ambos_enfoques():
    """Ejecuta ambos enfoques para mostrar la diferencia"""
    
    print("🔍 COMPARANDO AMBOS ENFOQUES")
    print("=" * 50)
    
    print("1️⃣ Sin structured output:")
    intentar_json_manual()
    
    print(f"\n{'='*50}")
    print("2️⃣ Con structured output:")
    usar_structured_output()
    
# Ejecutar comparación
comparar_ambos_enfoques()


  ### ✅ Conclusión:







  **Structured Output** resuelve todos los problemas:



  ✅ **Garantiza JSON válido siempre** - Sin errores de sintaxis



  ✅ **Compatible con Pydantic al 100%** - Tipos y estructura correctos



  ✅ **Sin texto extra ni problemas de formato** - Solo el JSON solicitado



  ✅ **Campos obligatorios siempre presentes** - Nunca faltan campos requeridos



  ✅ **Enums y tipos complejos validados** - Estructura anidada perfecta

  ## 4. Tool Calling 

  ### ¿Qué es Tool Calling?

  **Tool Calling** permite que el modelo use funciones Python que nosotros definamos



  📚 **Documentación útil:**

  - [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)

  - [Azure OpenAI Function Calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling)

In [ ]:
import json
from pydantic import BaseModel, Field
from openai import pydantic_function_tool

# Función simple de calculadora
def calcular(operacion: str, a: float, b: float) -> dict:
    """Realiza operaciones matemáticas básicas"""
    try:
        if operacion == "suma":
            resultado = a + b
        elif operacion == "resta":
            resultado = a - b
        elif operacion == "multiplicacion":
            resultado = a * b
        elif operacion == "division":
            if b == 0:
                return {"error": "No se puede dividir por cero"}
            resultado = a / b
        else:
            return {"error": f"Operación no válida: {operacion}"}
        
        return {
            "operacion": operacion,
            "numero1": a,
            "numero2": b,
            "resultado": resultado
        }
    except Exception as e:
        return {"error": f"Error en el cálculo: {str(e)}"}

def ejemplo_calculadora_tool_calling(message):
    """Ejemplo básico de tool calling con calculadora"""
    
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-02-01"
    )
    
    # Definir la herramienta de calculadora
    tools = [
        {
            "type": "function",
            "function": {
                "name": "CalculadoraTool",
                "description": "Realiza operaciones matemáticas básicas",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "operacion": {
                            "type": "string", 
                            "enum": ["suma", "resta", "multiplicacion", "division"],
                            "description": "Tipo de operación a realizar"
                        },
                        "a": {"type": "number", "description": "Primer número"},
                        "b": {"type": "number", "description": "Segundo número"}
                    },
                    "required": ["operacion", "a", "b"]
                }
            }
        }
    ]

    # Definir tools con pydantic_function_tool
    class Operacion(str, Enum):
        suma = "suma"
        resta = "resta"
        multiplicacion = "multiplicacion"
        division = "division"

    class CalculadoraTool(BaseModel):
        operacion: Operacion = Field(..., description="Tipo de operación a realizar")
        a: float = Field(..., description="Primer número")
        b: float = Field(..., description="Segundo número")

    tools = [pydantic_function_tool(CalculadoraTool)]


    
    print("CALCULADORA CON TOOL CALLING")
    print("=" * 40)
    
    # Pregunta que requiere cálculos
    mensajes = [
        {"role": "user", "content": message}
    ]
    
    try:
        # Primera llamada al modelo
        response = client.chat.completions.create(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
            messages=mensajes,
            tools=tools,
            tool_choice="auto" # auto|required|none
        )
        
        response_message = response.choices[0].message
        
        # ¿El modelo quiere usar herramientas?
        if response_message.tool_calls:
            print("🤖 El modelo detectó que necesita usar la calculadora")
            
            # Añadir mensaje del asistente
            mensajes.append(response_message)
            
            # Ejecutar cada cálculo que pidió el modelo
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"\n🔧 Ejecutando: {function_name}")
                print(f"📝 Con parámetros: {function_args}")
                
                # Llamar a la función de calculadora
                if function_name == "CalculadoraTool":
                    result = calcular(**function_args)
                else:
                    result = {"error": "Función no encontrada"}
                
                print(f"✅ Resultado: {result}")
                
                # Añadir el resultado al historial
                mensajes.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(result)
                })
            
            # Segunda llamada para que el modelo use los resultados
            final_response = client.chat.completions.create(
                model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
                messages=mensajes
            )
            
            print(f"\n🤖 Respuesta final del asistente:")
            print(final_response.choices[0].message.content)
            
        else:
            print(f"🤖 Respuesta directa: {response_message.content}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

# Ejecutar ejemplo
ejemplo_calculadora_tool_calling( "¿Cuánto es 25 + 17? Y también calcula 100 / 4")


  ### 💡 ¿Cómo funciona Tool Calling?







  El proceso es sencillo:



  1. **Definimos funciones Python** que el modelo puede usar



  2. **Describimos las funciones** en formato JSON Schema



  3. **El modelo decide** cuándo y cómo usar las funciones



  4. **Ejecutamos las funciones** con los parámetros que el modelo proporciona



  5. **Devolvemos los resultados** al modelo para que genere la respuesta final







  ### ✅ Ventajas del Tool Calling:



  - **Precisión**: El modelo no "adivina" los cálculos



  - **Funcionalidad**: Puede acceder a APIs, bases de datos, etc.



  - **Flexibilidad**: Múltiples herramientas en una conversación



  - **Confiabilidad**: Resultados exactos y validados

  ## 5. Bases de Datos con SQLAlchemy

  ### ¿Por qué SQLAlchemy?

  **SQLAlchemy** es el ORM más popular de Python que nos permite:

  - **Abstracción**: Trabajar con objetos Python en lugar de SQL

  - **Portabilidad**: Cambiar entre diferentes bases de datos fácilmente

  - **Seguridad**: Prevención automática de inyección SQL

  - **Productividad**: Menos código, más funcionalidad



  📚 **Documentación útil:**

  - [SQLAlchemy Documentation](https://docs.sqlalchemy.org/)

  - [SQLAlchemy Tutorial](https://docs.sqlalchemy.org/en/20/tutorial/)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import DeclarativeBase, sessionmaker

class Base(DeclarativeBase):
    pass

class Tarea(Base):
    """Una tarea simple con nombre y prioridad"""
    __tablename__ = 'tareas'
    
    id = Column(Integer, primary_key=True)
    nombre = Column(String(100))
    prioridad = Column(Integer, default=1)  # 1=baja, 2=media, 3=alta
    
    def __repr__(self):
        return f"<Tarea(nombre='{self.nombre}', prioridad={self.prioridad})>"

def ejemplo_completo_sqlalchemy():
    """Ejemplo completo: crear, leer, buscar, actualizar y borrar"""
    
    print("📚 EJEMPLO COMPLETO DE SQLALCHEMY")
    print("=" * 45)
    
    # Crear base de datos en archivo
    engine = create_engine("sqlite:///tareas.db", echo=False)
    Base.metadata.create_all(engine)
    print("✅ Tabla 'tareas' creada")
    
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # 1. CREAR - Añadir tareas
    tareas = [
        Tarea(nombre="Estudiar Python", prioridad=3),
        Tarea(nombre="Hacer ejercicio", prioridad=2),
        Tarea(nombre="Ver película", prioridad=1),
        Tarea(nombre="Hacer compras", prioridad=2)
    ]
    session.add_all(tareas)
    session.commit()
    print("✅ 4 tareas añadidas")
    
    # 2. LEER - Ver todas las tareas
    todas_tareas = session.query(Tarea).all()
    print(f"\n📋 Todas las tareas ({len(todas_tareas)}):")
    for tarea in todas_tareas:
        prioridad_text = {1: "Baja", 2: "Media", 3: "Alta"}[tarea.prioridad]
        print(f"   {tarea.id}. {tarea.nombre} (Prioridad: {prioridad_text})")
    
    # 3. BUSCAR - Solo tareas importantes
    tareas_importantes = session.query(Tarea).filter(Tarea.prioridad == 3).all()
    print(f"\n🔥 Tareas importantes ({len(tareas_importantes)}):")
    for tarea in tareas_importantes:
        print(f"   - {tarea.nombre}")
    
    # 4. ACTUALIZAR - Cambiar prioridad de compras
    tarea_compras = session.query(Tarea).filter(Tarea.nombre.like("%compras%")).first()
    if tarea_compras:
        tarea_compras.prioridad = 3
        session.commit()
        print(f"\n⬆️ '{tarea_compras.nombre}' ahora es prioridad alta")
    
    # 5. BORRAR - Eliminar tareas de baja prioridad
    tareas_baja = session.query(Tarea).filter(Tarea.prioridad == 1).all()
    for tarea in tareas_baja:
        print(f"🗑️ Borrando: {tarea.nombre}")
        session.delete(tarea)
    session.commit()
    
    # 6. CONTAR - ¿Cuántas quedan?
    total_final = session.query(Tarea).count()
    print(f"\n📊 Tareas restantes: {total_final}")
    
    session.close()

# Ejecutar ejemplo completo
ejemplo_completo_sqlalchemy()


  ### ⚠️ Importante: Cerrar sesiones







  **¿Por qué es importante `session.close()`?**



  1. **Liberar memoria**: Las sesiones mantienen objetos en memoria



  2. **Liberar conexiones**: Las bases de datos tienen límites de conexiones concurrentes



  3. **Evitar bloqueos**: Las sesiones pueden mantener "locks" en la base de datos



  4. **Buenas prácticas**: Limpiar recursos cuando terminas de usarlos

  ### 💡 Formas de manejar sesiones:







  **❌ Forma manual (propensa a errores):**

  ```python

  session = Session()

  try:

      # ... hacer operaciones

  except Exception:

      # Si algo falla aquí, ¡nunca llegaremos al close()!

      pass

  session.close()  # ¡Puede no ejecutarse nunca!

  ```



  **✅ Forma moderna (automática y segura):**

  ```python

  with Session() as session:

      # ... hacer operaciones

      # Se cierra SIEMPRE al salir del bloque, incluso si hay errores

  ```







  **🔒 Ventaja clave:** Los context managers (`with`) garantizan que la sesión se cierre **siempre**, incluso si ocurre una excepción dentro del bloque. Esto previene conexiones "colgadas" que pueden causar problemas.

In [ ]:
def demostrar_context_managers():
    """Ejemplo práctico de context managers"""
    
    engine = create_engine("sqlite:///demo.db", echo=False)
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    
    # Ejemplo práctico del context manager
    with Session() as session:
        tarea = Tarea(nombre="Probar context manager", prioridad=2)
        session.add(tarea)
        session.commit()
        print(f"✅ Tarea '{tarea.nombre}' añadida usando context manager")
        # session.close() no es necesario aquí - se hace automáticamente

# Ejecutar demostración
demostrar_context_managers()


  ## 6. Integración LLM y base de datos
  


### 6.1 Structured Output + Base de datos

  Vamos a combinar ambas tecnologías de forma práctica:

  1. **Input**: Usuario proporciona información en texto libre

  2. **Structured Output**: Modelo convierte el texto a datos estructurados

  3. **SQLAlchemy**: Guardamos los datos estructurados en la base de datos

In [ ]:
import os
import json
from pydantic import BaseModel, Field
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import DeclarativeBase, sessionmaker
from openai import AzureOpenAI
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Definir Base y modelo para esta sección
class Base(DeclarativeBase):
    pass

class Tarea(Base):
    """Una tarea con nombre, descripción y prioridad"""
    __tablename__ = 'tareas'
    
    id = Column(Integer, primary_key=True)
    nombre = Column(String(100))
    descripcion = Column(String(500))
    prioridad = Column(Integer)  # 1=baja, 2=media, 3=alta
    estimacion_horas = Column(Float)

# Clase Pydantic para validar los datos (mismos campos que la tabla)
class TareaEstructurada(BaseModel):
    nombre: str = Field(description="Nombre corto y descriptivo de la tarea")
    descripcion: str = Field(description="Descripción detallada de qué hay que hacer")
    prioridad: int = Field(description="Nivel de prioridad: 1=baja, 2=media, 3=alta", ge=1, le=3)
    estimacion_horas: float = Field(description="Horas estimadas para completar la tarea", gt=0)

def procesar_tarea_libre(texto_usuario: str):
    """Procesa texto libre y lo guarda estructurado en base de datos"""
    
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-08-01-preview"
    )
    
    print("🔄 INTEGRACIÓN: TEXTO → STRUCTURED OUTPUT → BASE DE DATOS")
    print("=" * 65)
    
    print("📝 Input del usuario:")
    print(f"   {texto_usuario.strip()}")
    
    # STRUCTURED OUTPUT: Convertir a datos estructurados
    prompt = f"""Analiza esta tarea y extrae la información estructurada:
    
    Texto: {texto_usuario}
    
    Extrae: nombre de la tarea, descripción, prioridad (1-3) y estimación de horas."""
    
    response = client.chat.completions.parse(
        model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
        messages=[{"role": "user", "content": prompt}],
        response_format=TareaEstructurada
    )
    
    # Obtener respuesta estructurada directamente
    tarea_estructurada = response.choices[0].message.parsed
    
    print(f"\n🎯 Datos estructurados extraídos:")
    print(f"   📋 Nombre: {tarea_estructurada.nombre}")
    print(f"   📄 Descripción: {tarea_estructurada.descripcion}")
    print(f"   ⚡ Prioridad: {tarea_estructurada.prioridad}/3")
    print(f"   ⏱️ Estimación: {tarea_estructurada.estimacion_horas} horas")
    
    # SQLALCHEMY: Guardar en base de datos
    engine = create_engine("sqlite:///tareas_estructuradas.db", echo=False)
    Base.metadata.create_all(engine)
    
    with sessionmaker(bind=engine)() as session:
        nueva_tarea = Tarea(
            nombre=tarea_estructurada.nombre,
            descripcion=tarea_estructurada.descripcion,
            prioridad=tarea_estructurada.prioridad,
            estimacion_horas=tarea_estructurada.estimacion_horas
        )
        session.add(nueva_tarea)
        session.commit()
        
        print(f"\n💾 Tarea guardada en base de datos con ID: {nueva_tarea.id}")
    
    print(f"\n✅ Flujo completo exitoso:")
    print(f"   📝 Texto libre → 🎯 Datos estructurados → 💾 Base de datos")

# Ejecutar con texto de ejemplo
texto_ejemplo = """
Necesito preparar una presentación sobre inteligencia artificial para el trabajo.
Es bastante urgente porque la tengo que entregar la próxima semana.  
Creo que me llevará unas 6 horas completarla entre investigación y diseño.
"""

procesar_tarea_libre(texto_ejemplo)


  ### 6.2 Tool Calling + base de datos

  El mismo ejemplo pero usando **tool calling** para que el modelo decida cuándo guardar:

  1. **Input**: Usuario proporciona información en texto libre

  2. **Tool Calling**: El modelo decide si quiere o no guardar el resultado

  3. **SQLAlchemy**: Guardamos los datos estructurados en la base de datos

In [ ]:

from openai import pydantic_function_tool

def guardar_tarea_estructurada(nombre: str, descripcion: str, prioridad: int, estimacion_horas: float) -> dict:
    """Herramienta para guardar una tarea en la base de datos"""
    try:
        engine = create_engine("sqlite:///tareas_tools.db", echo=False)
        Base.metadata.create_all(engine)
        
        with sessionmaker(bind=engine)() as session:
            nueva_tarea = Tarea(
                nombre=nombre, 
                descripcion=descripcion,
                prioridad=prioridad,
                estimacion_horas=estimacion_horas
            )
            session.add(nueva_tarea)
            session.commit()
            
            return {
                "exito": True,
                "mensaje": f"Tarea '{nombre}' guardada con ID {nueva_tarea.id}",
                "id": nueva_tarea.id
            }
    except Exception as e:
        return {"exito": False, "error": str(e)}

def procesar_tarea_con_tools(texto_usuario: str):
    """Usa tool calling para procesar y guardar tareas"""
    
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-02-01"
    )
    
    # Definir tools con pydantic_function_tool
    
    class GuardarTarea(BaseModel):
        nombre: str = Field(..., description="Nombre de la tarea")
        descripcion: str = Field(..., description="Descripción detallada de la tarea")
        prioridad: int = Field(..., description="Prioridad de la tarea (1-3)", ge=1, le=3)
        estimacion_horas: float = Field(..., description="Horas estimadas para completar la tarea", gt=0)
    
    tools = [pydantic_function_tool(GuardarTarea)]

    
    print("🔧 INTEGRACIÓN CON TOOL CALLING")
    print("=" * 40)
    
    print(f"📝 Input: {texto_usuario.strip()}")
    
    # El modelo procesa el texto y decide usar la herramienta
    mensajes = [{
        "role": "user", 
        "content": texto_usuario
    }]
    
    response = client.chat.completions.create(
        model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
        messages=mensajes,
        tools=tools,
        tool_choice="auto"
    )
    
    # Ejecutar tool call si el modelo decide usarlo
    if response.choices[0].message.tool_calls:
        mensajes.append(response.choices[0].message)
        
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"\n🤖 El modelo decidió usar la herramienta: {function_name}")
            print(f"   Parámetros: {function_args}")
            
            if tool_call.function.name == "GuardarTarea":

                resultado = guardar_tarea_estructurada(**function_args)
                print(f"\n💾 {resultado['mensaje']}")

                # Añadir el resultado al historial
                mensajes.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(resultado)
                })
            
            # Segunda llamada para que el modelo use los resultados
            print(mensajes)
            final_response = client.chat.completions.create(
                model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
                messages=mensajes
            )
            
            print(f"\n🤖 Respuesta final del asistente:")
            print(final_response.choices[0].message.content)
        
                
    else:
        print("🤖 El modelo no decidió usar herramientas")
        print(f"🤖 Respuesta directa del modelo: {response.choices[0].message.content}")
    


# Ejecutar con el mismo texto
texto_ejemplo = """Necesito preparar una presentación sobre inteligencia artificial para el trabajo.
Es bastante urgente porque la tengo que entregar la próxima semana.  
Creo que me llevará unas 6 horas completarla entre investigación y diseño.
"""
procesar_tarea_con_tools(texto_ejemplo)


  ### 🤔 ¿Cuándo usar cada enfoque?

  **🎯 Usa Structured Output cuando:**

  - Necesites **formato específico garantizado**

  - Quieras **control total del proceso**

  - El flujo sea **predecible y simple**


  **🔧 Usa Tool Calling cuando:**

  - El modelo deba **decidir qué hacer**

  - Tengas **múltiples opciones/herramientas**

  - Quieras **conversaciones**

  - Necesites **flexibilidad en las acciones**


  **💡 En la práctica:**

  - **Structured Output**: "Convierte este texto a formato específico"

  - **Tool Calling**: "Analiza este texto y haz lo que creas necesario"







  ## 7. Problema: Asistente Financiero con Sistema de Notas

  ### 🎯 Objetivo

  Implementar la funcionalidad **faltante** de borrar notas en un asistente financiero completo que ya incluye:

  - ✅ Tool calling para crear y leer notas

  - ✅ Base de datos SQLAlchemy para persistencia

  - ❌ **FALTA**: Funcionalidad para borrar notas



  ### 📋 Tu tarea

  El código siguiente es un asistente financiero **casi completo**. Solo falta implementar:

  1. **Tool definition**: Añadir la definición de la herramienta `borrar_nota`

  2. **Tool execution**: Implementar la ejecución de `borrar_nota` en el método `ejecutar_tool_call`

  3. **Database method**: Implementar el método `borrar_nota` en la clase `NotasDB`







  ### 💡 Pistas

  - La herramienta debe recibir un `nota_id` (integer) como parámetro

  - Sigue el mismo patrón que las otras herramientas

In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import AzureOpenAI
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, func
from sqlalchemy.orm import DeclarativeBase, sessionmaker, Session
from typing import List, Optional

# Cargar variables de entorno
load_dotenv()

# Configuración de SQLAlchemy
class Base(DeclarativeBase):
    pass

class Nota(Base):
    """Modelo de la tabla notas"""
    __tablename__ = 'notas'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    titulo = Column(String(255), nullable=False)
    contenido = Column(Text, nullable=False)
    categoria = Column(String(50), nullable=False, default='general')
    fecha_creacion = Column(DateTime, default=func.now())
    fecha_modificacion = Column(DateTime, default=func.now(), onupdate=func.now())
    
    def __repr__(self):
        return f"<Nota(id={self.id}, titulo='{self.titulo[:30]}...', categoria='{self.categoria}')>"
    
    def to_dict(self):
        """Convierte la nota a diccionario"""
        return {
            'id': self.id,
            'titulo': self.titulo,
            'contenido': self.contenido,
            'categoria': self.categoria,
            'fecha_creacion': self.fecha_creacion.isoformat() if self.fecha_creacion else None,
            'fecha_modificacion': self.fecha_modificacion.isoformat() if self.fecha_modificacion else None
        }

class NotasDB:
    """Clase para manejar la base de datos de notas con SQLAlchemy"""
    
    def __init__(self, db_url: str = "sqlite:///notas.db"):
        self.engine = create_engine(db_url, echo=False)
        self.SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=self.engine)
        self.init_db()
    
    def init_db(self):
        """Crear las tablas si no existen"""
        Base.metadata.create_all(bind=self.engine)
    
    def get_session(self) -> Session:
        """Obtener una sesión de base de datos"""
        return self.SessionLocal()
    
    def crear_nota(self, titulo: str, contenido: str, categoria: str = "general") -> int:
        """Crear una nueva nota con validación"""
        # Validaciones
        if not titulo or not titulo.strip():
            raise ValueError("El título no puede estar vacío")
        if not contenido or not contenido.strip():
            raise ValueError("El contenido no puede estar vacío")
        if len(titulo) > 255:
            raise ValueError("El título no puede exceder 255 caracteres")
        
        titulo = titulo.strip()
        contenido = contenido.strip()
        categoria = categoria.strip().lower()
        
        try:
            with self.get_session() as session:
                nueva_nota = Nota(
                    titulo=titulo,
                    contenido=contenido,
                    categoria=categoria
                )
                session.add(nueva_nota)
                session.commit()
                session.refresh(nueva_nota)
                return nueva_nota.id
        except Exception as e:
            raise ValueError(f"Error creando nota: {str(e)}")
    
    def leer_notas(self) -> List[Nota]:
        """Leer todas las notas ordenadas por fecha de creación descendente"""
        try:
            with self.get_session() as session:
                return session.query(Nota).order_by(Nota.fecha_creacion.desc()).all()
        except Exception as e:
            raise ValueError(f"Error leyendo notas: {str(e)}")
    
    def contar_notas(self) -> int:
        """Contar total de notas"""
        try:
            with self.get_session() as session:
                return session.query(Nota).count()
        except Exception as e:
            raise ValueError(f"Error contando notas: {str(e)}")
    

class ChatbotFinanciero:
    """Asistente financiero con sistema de notas usando Azure OpenAI y SQLAlchemy"""

    NUMERO_MAXIMO_MENSAJES = 20
    MAX_ITERACIONES_TOOLS = 3  # Máximo número de iteraciones con tools
    
    CATEGORIAS_VALIDAS = [
        "finanzas", "presupuesto", "inversiones", "gastos", 
        "ingresos", "objetivos", "deudas", "ahorros", "general"
    ]
    
    def __init__(self, db_url: str = "sqlite:///notas.db"):
        """Inicializar el chatbot"""
        prompt_sistema = """Eres un asesor financiero experto y profesional. 
        Ayudas a las personas a tomar decisiones financieras inteligentes.
        Eres amigable pero profesional, y siempre ofreces consejos prácticos y éticos.
        Mantén tus respuestas concisas pero informativas.
        
        Tienes acceso a un sistema de notas donde puedes crear y consultar información importante
        sobre las finanzas del usuario. Usa estas herramientas cuando sea apropiado para ofrecer
        un servicio más personalizado y recordar información relevante entre conversaciones.
        
        Categorías disponibles para notas: finanzas, presupuesto, inversiones, gastos, ingresos, 
        objetivos, deudas, ahorros, general."""
        
        self.mensajes = [{"role": "system", "content": prompt_sistema}]
        self.notas_db = NotasDB(db_url)
        self.client = self._init_azure_client()
        self.deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
        self.estadisticas = {
            'mensajes_enviados': 0,
            'tool_calls_ejecutados': 0,
            'iteraciones_tools': 0,
            'inicio_conversacion': datetime.now()
        }

    def _init_azure_client(self):
        """Inicializa el cliente de Azure OpenAI"""
        try:
            return AzureOpenAI(
                api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                api_version="2024-02-01"
            )
        except Exception as e:
            print(f"❌ Error inicializando Azure OpenAI: {e}")
            print("💡 Verifica tus variables de entorno:")
            print("   - AZURE_OPENAI_API_KEY")
            print("   - AZURE_OPENAI_ENDPOINT") 
            print("   - AZURE_OPENAI_DEPLOYMENT_NAME")
            raise e

    def _get_tools_definition(self):
        """Define las herramientas disponibles para tool calling"""
        return [
            {
                "type": "function",
                "function": {
                    "name": "crear_nota",
                    "description": "Crea una nueva nota financiera con información importante del usuario",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "titulo": {
                                "type": "string",
                                "description": "Título descriptivo y claro de la nota"
                            },
                            "contenido": {
                                "type": "string",
                                "description": "Contenido detallado de la nota con información relevante"
                            },
                            "categoria": {
                                "type": "string",
                                "description": "Categoría de la nota",
                                "enum": self.CATEGORIAS_VALIDAS,
                                "default": "general"
                            }
                        },
                        "required": ["titulo", "contenido"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "leer_notas",
                    "description": "Lee todas las notas guardadas para consultar información previa del usuario",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            },
        ]

    def ejecutar_tool_call(self, function_name: str, arguments: dict) -> str:
        """Ejecuta las funciones llamadas por el modelo"""
        try:
            self.estadisticas['tool_calls_ejecutados'] += 1
            
            if function_name == "crear_nota":
                nota_id = self.notas_db.crear_nota(
                    titulo=arguments["titulo"],
                    contenido=arguments["contenido"],
                    categoria=arguments.get("categoria", "general")
                )
                return f"✅ Nota creada exitosamente con ID: {nota_id}"
            
            elif function_name == "leer_notas":
                notas = self.notas_db.leer_notas()
                if notas:
                    resultado = []
                    for nota in notas:
                        resultado.append(nota.to_dict()) 
                    
                    return f"📋 {len(resultado)} notas encontradas:\n" + json.dumps(
                        resultado, ensure_ascii=False, indent=2
                    )
                else:
                    return "📭 No hay notas guardadas aún."
            
            else:
                return f"❌ Función no reconocida: {function_name}"
        
        except Exception as e:
            return f"❌ Error ejecutando {function_name}: {str(e)}"

    def _agregar_al_historial(self, mensaje: dict):
        """Método centralizado para agregar mensajes al historial"""
        self.mensajes.append(mensaje)
        
    def _procesar_tool_calls(self, tool_calls):
        """Procesa tool calls y los agrega al historial"""
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Ejecutando: {function_name}")
            function_response = self.ejecutar_tool_call(function_name, function_args)
            self.estadisticas['iteraciones_tools'] += 1
            
            tool_message = {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
            
            self._agregar_al_historial(tool_message)

                
    def responder(self, mensaje: str) -> str:
        """
        Método principal que genera una respuesta, gestionando el bucle de tool calling
        de forma unificada.
        """
        try:
            # 1. Agregar mensaje del usuario al historial
            self._agregar_al_historial({"role": "user", "content": mensaje})
            self.estadisticas['mensajes_enviados'] += 1

            # 2. Bucle unificado
            # Se ejecuta N veces para tools + 1 vez para la respuesta final forzada si es necesario
            for iteracion in range(self.MAX_ITERACIONES_TOOLS + 1):
                
                # En la última iteración posible, forzamos una respuesta sin herramientas
                if iteracion == self.MAX_ITERACIONES_TOOLS:
                    print("🔒 Límite de iteraciones alcanzado. Forzando respuesta final.")
                    tool_choice = "none"
                else:
                    tool_choice = "auto"

                response = self.client.chat.completions.create(
                    model=self.deployment_name,
                    messages=self.mensajes,
                    tools=self._get_tools_definition(),
                    tool_choice=tool_choice, 
                    temperature=0.0
                )
                response_message = response.choices[0].message
                self._agregar_al_historial(response_message.model_dump())

                # Si hay tool calls y NO estamos en la última iteración forzada...
                if response_message.tool_calls:
                    print(f"🔄 Procesando {len(response_message.tool_calls)} herramienta(s)...")
                    self._procesar_tool_calls(response_message.tool_calls)
                    # Continuamos al siguiente ciclo para que el modelo evalúe el resultado
                    continue

                # Si llegamos aquí, es porque el modelo ha generado una respuesta de texto
                # (ya sea por decisión propia o porque le hemos forzado en la última iteración).
                # Esta es la respuesta definitiva.
                respuesta_final = response_message.content 

                print(f"\n🤖 Asistente: {respuesta_final}")
                return respuesta_final

            return "❌ No se pudo generar una respuesta"

        except Exception as e:
            print(f"❌ Error al responder: {e}")
            raise ValueError(f"❌ Error al responder: {str(e)}")
    
    def mostrar_estadisticas(self) -> None:
        """Muestra estadísticas de la conversación"""
        try:
            duracion = datetime.now() - self.estadisticas['inicio_conversacion']
            total_notas = self.notas_db.contar_notas()
            
            # Contar mensajes por tipo
            mensajes_usuario = len([m for m in self.mensajes if m['role'] == 'user'])
            mensajes_asistente = len([m for m in self.mensajes if m['role'] == 'assistant'])
            mensajes_tool = len([m for m in self.mensajes if m['role'] == 'tool'])
            
            print("\n📊 Estadísticas de la conversación:")
            print("=" * 50)
            print(f"   ⏱️  Duración: {duracion.seconds} segundos")
            print(f"   📝 Mensajes enviados: {self.estadisticas['mensajes_enviados']}")
            print(f"   🔧 Herramientas ejecutadas: {self.estadisticas['tool_calls_ejecutados']}")
            print(f"   🔄 Iteraciones con tools: {self.estadisticas['iteraciones_tools']}")
            print(f"   💬 Total en historial: {len(self.mensajes) - 1}")
            print(f"      👤 Usuario: {mensajes_usuario}")
            print(f"      🤖 Asistente: {mensajes_asistente}")
            print(f"      🔧 Herramientas: {mensajes_tool}")
            print(f"   📋 Total notas creadas: {total_notas}")
            
        except Exception as e:
            print(f"❌ Error mostrando estadísticas: {e}")
    
    def limpiar_historial(self):
        """Limpia el historial de conversación"""
        try:
            prompt_sistema = self.mensajes[0]
            self.mensajes = [prompt_sistema]
            print("🧹 Historial de conversación limpiado")
        except Exception as e:
            print(f"❌ Error limpiando historial: {e}")

    def mostrar_ayuda(self):
        """Muestra la ayuda con los comandos disponibles"""
        print("\n💡 Comandos disponibles:")
        print("=" * 50)
        print("   /estadisticas - Mostrar estadísticas de la sesión")
        print("   /limpiar      - Limpiar historial de conversación")
        print("   /notas        - Mostrar todas las notas guardadas")
        print("   /ayuda        - Mostrar esta ayuda")
        print("   /salir        - Terminar conversación")
        print()
        print("🔧 Herramientas automáticas del asistente:")
        print("   • Crear notas financieras categorizadas")
        print("   • Consultar todas las notas guardadas")
        print(f"   • Máximo {self.MAX_ITERACIONES_TOOLS} iteraciones con herramientas por mensaje")
        print()
        print(f"📂 Categorías disponibles: {', '.join(self.CATEGORIAS_VALIDAS)}")

    def mostrar_notas(self):
        """Muestra todas las notas guardadas de forma legible"""
        try:
            notas = self.notas_db.leer_notas()
            if notas:
                print(f"\n📋 Todas las notas guardadas ({len(notas)} total):")
                print("=" * 60)
                
                # Agrupar por categoría
                notas_por_categoria = {}
                for nota in notas:
                    categoria = nota.categoria
                    if categoria not in notas_por_categoria:
                        notas_por_categoria[categoria] = []
                    notas_por_categoria[categoria].append(nota)
                
                for categoria, notas_categoria in notas_por_categoria.items():
                    print(f"\n📂 {categoria.upper()} ({len(notas_categoria)} notas):")
                    print("-" * 40)
                    for nota in notas_categoria[:5]:  # Mostrar máximo 5 por categoría
                        print(f"   ID: {nota.id} | {nota.titulo}")
                        print(f"   Fecha: {nota.fecha_creacion}")
                        contenido = nota.contenido[:150] + "..." if len(nota.contenido) > 150 else nota.contenido
                        print(f"   Contenido: {contenido}")
                        print()
                    
                    if len(notas_categoria) > 5:
                        print(f"   ... y {len(notas_categoria) - 5} notas más en esta categoría")
                    print()
            else:
                print("📝 No hay notas guardadas aún.")
                print("💡 El asistente creará notas automáticamente cuando sea relevante.")
        except Exception as e:
            print(f"❌ Error mostrando notas: {e}")

def main():
    """Función principal para el chatbot interactivo"""
    print("🤖 Asistente Financiero Inteligente")
    print("🔧 Powered by Azure OpenAI + SQLAlchemy")
    print("=" * 60)
    
    # Permitir configuración de base de datos
    db_url = os.getenv('DATABASE_URL', 'sqlite:///notas.db')
    print(f"🗄️ Base de datos: {db_url}")
    
    try:
        # Crear instancia del chatbot
        chatbot = ChatbotFinanciero(db_url)
        
        # Verificar conexión y mostrar estadísticas iniciales
        print("🔍 Verificando conexión con Azure OpenAI...")
        notas_existentes = chatbot.notas_db.contar_notas()
        
        print("✅ Conexión establecida exitosamente")
        print(f"📋 Base de datos inicializada: {notas_existentes} notas existentes")
        
        print(f"\n🔄 Configuración del sistema de herramientas:")
        print(f"   • Máximo {chatbot.MAX_ITERACIONES_TOOLS} iteraciones por mensaje")
        print(f"   • 2 herramientas disponibles: crear_nota y leer_notas")

        print(f"   • Respuesta final forzada en la iteración {chatbot.MAX_ITERACIONES_TOOLS}")
        
        print("\n💡 Funciones disponibles:")
        print("   • Asesoramiento financiero personalizado")
        print("   • Creación automática de notas categorizadas")
        print("   • Consulta de información previa guardada")
        print("   • Memoria persistente entre conversaciones")
        
        print("\n💬 Escribe /ayuda para ver todos los comandos disponibles")
        print("=" * 60)
        
        # Bucle principal del chat
        while True:
            user_input = input("\n👤 Tú: ").strip()
            
            if user_input.lower() == '/salir':
                chatbot.mostrar_estadisticas()
                print("\n👋 ¡Gracias por usar el asistente financiero!")
                print("💾 Todas tus notas han sido guardadas para futuras consultas.")
                break
            
            if not user_input:
                continue
            
            # Procesar comandos especiales
            if user_input == '/estadisticas':
                chatbot.mostrar_estadisticas()
            elif user_input == '/limpiar':
                chatbot.limpiar_historial()
            elif user_input == '/notas':
                chatbot.mostrar_notas()
            elif user_input == '/ayuda':
                chatbot.mostrar_ayuda()
            else:
                try:
                    chatbot.responder(user_input)
                except Exception as e:
                    print(f"❌ Error procesando mensaje: {e}")
                    print("💡 Intenta de nuevo o escribe /ayuda para ver los comandos disponibles")

    except Exception as e:
        print(f"❌ Error crítico: {e}")
        print("💡 Verifica tu configuración de Azure OpenAI y las variables de entorno")

if __name__ == "__main__":
    main()

  ## 🎉 ¡Felicitaciones!


  Has completado el lab de **Structured Output, Tool Calling y Gestión de Bases de Datos**. Ahora dominas:

  ✅ **Structured Output**: Respuestas estructuradas y validadas con Pydantic

  ✅ **Tool Calling**: Integración de herramientas externas en conversaciones

  ✅ **SQLAlchemy**: Gestión profesional de bases de datos relacionales

  ✅ **Integración completa**: Combinación de todas las tecnologías

  ✅ **Cuándo usar cada enfoque**: Criterios para elegir la técnica adecuada